In [ ]:
!pip install pyspark


     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 39.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=f86a67de62b3a0ebee8524059651d73ac2ed5540b1c06a10f42952e981b7ace5
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


SparkSession

In [ ]:
# https://towardsdatascience.com/sparksession-vs-sparkcontext-vs-sqlcontext-vs-hivecontext-741d50c9486a

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

app_name = 'DataEngineering'

spark = SparkSession \
      .builder \
      .enableHiveSupport() \
      .config("hive.exec.dynamic.partition", "true") \
      .config("hive.exec.dynamic.partition.mode", "nonstrict") \
      .appName(app_name) \
      .getOrCreate()\


sc = spark.sparkContext

Operations on RDDs

*Fake Friends*


---


fakefriends.csv schema:

      id
      name
      age
      number_of_friends (on social media)


დავთვალოთ თითოეული ასაკისთვის სოციალურ ქსელში მეგობრების საშუალო რაოდენობა RDD-ების და Map/MapValues ფუნქციის გამოყენებით

In [ ]:
def parseLine(line):
    fields = line.split(',')
    age = int(fields[2])
    numFriends = int(fields[3])
    return (age, numFriends)

lines = sc.textFile('fakefriends.csv')

rdd = lines.map(parseLine)

totals_by_age = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

totals_by_age.collect()

avg_by_age = totals_by_age.mapValues(lambda x: x[0] / x[1])
# avg_by_age = totals_by_age.map(lambda x: (x[0], x[1][0] / x[1][1]))

results = avg_by_age.collect()

for result in results:
    print(result)

(26, 242.05882352941177)
(40, 250.8235294117647)
(68, 269.6)
(54, 278.0769230769231)
(38, 193.53333333333333)
(56, 306.6666666666667)
(36, 246.6)
(22, 206.42857142857142)
(60, 202.71428571428572)
(30, 235.8181818181818)
(42, 303.5)
(48, 281.4)
(50, 254.6)
(32, 207.9090909090909)
(58, 116.54545454545455)
(64, 281.3333333333333)
(52, 340.6363636363636)
(24, 233.8)
(20, 165.0)
(62, 220.76923076923077)
(44, 282.1666666666667)
(28, 209.1)
(66, 276.44444444444446)
(46, 223.69230769230768)
(18, 343.375)
(34, 245.5)
(33, 325.3333333333333)
(55, 295.53846153846155)
(59, 220.0)
(37, 249.33333333333334)
(27, 228.125)
(53, 222.85714285714286)
(57, 258.8333333333333)
(43, 230.57142857142858)
(35, 211.625)
(45, 309.53846153846155)
(67, 214.625)
(19, 213.27272727272728)
(51, 302.14285714285717)
(25, 197.45454545454547)
(21, 350.875)
(49, 184.66666666666666)
(39, 169.28571428571428)
(31, 267.25)
(41, 268.55555555555554)
(69, 235.2)
(65, 298.2)
(61, 256.22222222222223)
(29, 215.91666666666666)
(47, 233

*Temperatures in 1800*


---


temperatures.csv schema:

      station_id
      date
      entry_type
      degrees (C)


დავთვალოთ თითოეული მეტეოროლოგიური სადგურისთვის დაფიქსირებული უდაბლესი ტემპერატურა (entry_type = 'TMIN')

In [ ]:
from pyspark import SparkConf, SparkContext

def parseLine(line):
    fields = line.split(',')
    station_id = fields[0]
    entry_type = fields[2]
    temperature = float(fields[3])
    return (station_id, entry_type, temperature)

lines = sc.textFile('temperatures.csv')
parsedLines = lines.map(parseLine)

minTemps = parsedLines.filter(lambda x: 'TMIN' in x[1])
stationTemps = minTemps.map(lambda x: (x[0], x[2]))
minTemps = stationTemps.reduceByKey(lambda x, y: min(x,y))
results = minTemps.collect();

for result in results:
    print(result[0] + '\t{:.2f}C'.format(result[1]))


ITE00100554	-14.80C
EZE00100082	-13.50C


*Word Count*


---

book.txt ფაილში დავთვალოთ თითოეული სიტყვა რამდენჯერ გვხვდება.

In [ ]:
'''#1 CountByValue'''

import re

book = sc.textFile("book.txt")
words = book.flatMap(lambda x: x.split())
word_counts = words.countByValue()

for word, count in word_counts.items():
        print(word + " " + str(count))

''' convert dict to rdd '''
dict_to_rdd = sc.parallelize(list(word_counts.items()))
# dict_to_rdd.collect()

In [ ]:
# თან დავსორტოთ კლებადობით და ტოპ 10 ვნახოთ
'''#2 Map/ReduceByKey'''

import re

book = sc.textFile("book.txt")
words = book.flatMap(lambda x: x.split())

word_counts = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

sorted = word_counts.map(lambda x: (x[1], x[0])).sortByKey(ascending=False)
results = sorted.collect()[:10]

for result in results:
    count = str(result[0])
    word = result[1]
    print(word + "   " + count)


to   1789
your   1339
you   1267
the   1176
a   1148
of   941
and   901
that   641
in   552
is   531


Operations on DataFrames

*Fake Friends*


---


fakefriends.csv schema:

      id
      name
      age
      number_of_friends (on social media)


დავთვალოთ თითოეული ასაკისთვის სოციალურ ქსელში მეგობრების საშუალო რაოდენობა Spark-ის დატაფრეიმების გამოყენებით

In [ ]:
from pyspark.sql.types import *
import pyspark.sql.functions as f

friends = spark\
        .read\
        .option('header', 'false')\
        .option('inferSchema', 'true')\
        .csv('fakefriends.csv')\
        .toDF('id', 'name', 'age', 'number_of_friends')

friends\
  .groupBy('age')\
  .mean('number_of_friends')\
  .show()

print('ან მეორენაირად: ')

friends\
  .groupBy('age')\
  .agg(f.avg('number_of_friends').cast('decimal(10, 2)').alias('avg_num_of_friends'))\
  .orderBy('age', ascending=True) \
  .show(20)

+---+----------------------+
|age|avg(number_of_friends)|
+---+----------------------+
| 31|                267.25|
| 65|                 298.2|
| 53|    222.85714285714286|
| 34|                 245.5|
| 28|                 209.1|
| 26|    242.05882352941177|
| 27|               228.125|
| 44|     282.1666666666667|
| 22|    206.42857142857142|
| 47|    233.22222222222223|
| 52|     340.6363636363636|
| 40|     250.8235294117647|
| 20|                 165.0|
| 57|     258.8333333333333|
| 54|     278.0769230769231|
| 48|                 281.4|
| 19|    213.27272727272728|
| 64|     281.3333333333333|
| 41|    268.55555555555554|
| 43|    230.57142857142858|
+---+----------------------+
only showing top 20 rows

ან მეორენაირად: 
+---+------------------+
|age|avg_num_of_friends|
+---+------------------+
| 18|            343.38|
| 19|            213.27|
| 20|            165.00|
| 21|            350.88|
| 22|            206.43|
| 23|            246.30|
| 24|            233.80|
| 25|      

In [ ]:
friends\
  .select('name', 'age', 'number_of_friends')\
  .limit(10)\
  .show(truncate=0, vertical=True)

-RECORD 0---------------------
 name              | Will     
 age               | 33       
 number_of_friends | 385      
-RECORD 1---------------------
 name              | Jean-Luc 
 age               | 26       
 number_of_friends | 2        
-RECORD 2---------------------
 name              | Hugh     
 age               | 55       
 number_of_friends | 221      
-RECORD 3---------------------
 name              | Deanna   
 age               | 40       
 number_of_friends | 465      
-RECORD 4---------------------
 name              | Quark    
 age               | 68       
 number_of_friends | 21       
-RECORD 5---------------------
 name              | Weyoun   
 age               | 59       
 number_of_friends | 318      
-RECORD 6---------------------
 name              | Gowron   
 age               | 37       
 number_of_friends | 220      
-RECORD 7---------------------
 name              | Will     
 age               | 54       
 number_of_friends | 307      
-RECORD 

*Breaking News*


---


დატასეტში bbc_news.csv მოცემული გვაქვს 1500-მდე მოკლე სტატია BBC News-იდან. თითოეულ სტატიას აქვს უნიკალური ID და კატეგორიის label-ი (tech, business, politics, entertainment, sport).

მიზანია, რომ თითოეულ სტატიას დავუგენერიროთ tag-ები შემდეგი მარტივი კრიტერიუმით:  უნდა მოვძებნოთ 3 ყველაზე ხშირი სიტყვა ამ სტატიაში. აუცილებელია, რომ თითოეული სიტყვა მინიმუმ 10-ჯერ მაინც გვხვდებოდეს კონკრეტულ სტატიაში.

გაითვალისწინეთ, რომ:
 * Tag-ები არ უნდა იყოს ე.წ. stopword-ები ან სიმბოლოები. მათი ჩამონათვალი არის ფაილში stopwords.txt;
 * Tag-ები არ შეიძლება იყოს რიცხვები;
 * Tag-ები არ შეიძლება იყოს ერთ ან ორ სიმბოლოიანი სიტყვები;
 * სიტყვებად ჩავთვალოთ შემდეგი სიმბოლოებით დაყოფილი ტოკენები: 

          whitespace

          ,

          :

          !

          ?


საბოლოო resultset უნდა ჩაწეროთ csv ფაილად და უნდა შედგებოდეს ველებისგან: ***ArticleID, Category, Tag, Frequency***


In [ ]:
from pyspark.sql.functions import col, split, lit, array_except, length, size, array, explode, count, array_remove
from pyspark.sql.types import *

schema = StructType([ \
        StructField("ArticleID",IntegerType(),True), \
        StructField("Text",StringType(),True), \
        StructField("Category",StringType(),True)
      ])

news = spark.read.option('inferSchema', 'false').schema(schema).csv('bbc_news.csv')
stopwords = spark.read.option('header', 'true').csv('stopwords.txt')

''' array_except დუბლირებებს შლის, რაც არ გვაწყობს '''
# filtered = news\
#             .withColumn('Words', split(col('Text'), ' |\.|,|\:|\!|\?'))\
#             .withColumn('OriginalNumOfWords', lit(size(col('Words'))))\
#             .withColumn('WordsFiltered', array_except(col('Words'), array(*map(lit, stopwords))))\
#             .withColumn('ExceptStopwords', lit(size(col('WordsFiltered'))))\

# filtered.show(1, 0, True)

exploded = news\
            .withColumn('Words', split(col('Text'), ' |\.|,|\:|\!|\?'))\
            .select(news.ArticleID, news.Category, explode(col('Words')).alias('Word'))\
            .filter(length(col('Word')) > 2)\
            .where("Word != '' ")\

drop_nums = exploded\
            .withColumn('IsNumber', col('Word').cast('int').isNotNull())\
            .where(~col('IsNumber'))\
            .drop('IsNumber')\

filter_stop_words = drop_nums.alias('w')\
                      .join(stopwords.alias('s'), col('Word') == col('i'), 'leftanti')

filter_stop_words.show(10)

+---------+--------+---------+
|ArticleID|Category|     Word|
+---------+--------+---------+
|     1833|business| worldcom|
|     1833|business|  ex-boss|
|     1833|business| launches|
|     1833|business|  defence|
|     1833|business|  lawyers|
|     1833|business|defending|
|     1833|business|   former|
|     1833|business| worldcom|
|     1833|business|    chief|
|     1833|business|   bernie|
+---------+--------+---------+
only showing top 10 rows



In [ ]:
frequencies = filter_stop_words\
              .groupby('ArticleID', 'Category', 'Word')\
              .agg(count('*').alias('Freq'))\
              .where('Freq > 10')

frequencies.show()

+---------+-------------+--------+----+
|ArticleID|     Category|    Word|Freq|
+---------+-------------+--------+----+
|      760|     business|    fiat|  16|
|     1183|entertainment|   music|  13|
|      484|     business|  glazer|  12|
|     1229|         tech|  search|  14|
|       20|         tech|argonaut|  12|
|      435|     business|    card|  12|
|      623|     politics|   trust|  14|
|     1149|     politics|   local|  11|
|     1928|        sport|   point|  18|
|     2130|     business|  dollar|  11|
|      701|     politics|  police|  13|
|     1149|     politics|    said|  13|
|     1976|         tech|   phone|  11|
|      840|         tech|   firms|  11|
|     2115|entertainment| academy|  12|
|     1597|entertainment|   chart|  16|
|      156|entertainment|     lee|  15|
|      860|         tech|  mobile|  12|
|     1624|         tech|bloggers|  11|
|     1455|         tech|   yahoo|  24|
+---------+-------------+--------+----+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window


results = frequencies\
            .withColumn('rank', row_number()\
                        .over(
                                Window.partitionBy('ArticleID', 'Category').orderBy(col('Freq').desc())
                              ))\
            .where('rank <= 3')\
            .withColumnRenamed('Freq', 'Frequency')\
            .withColumnRenamed('Word', 'Tag')\
            .select('ArticleID', 'Category', 'Tag', 'Frequency')

Save to various file formats

In [ ]:
# save to CSV
results.write.mode('overwrite').csv('frequencies_all.csv')

In [ ]:
''' save to Parquet '''

results.write.mode('overwrite').parquet('frequencies_parquet.parquet')

# spark.read.parquet('frequencies_parquet.parquet').show()

In [ ]:
''' Save to ORC and partition by Category ''' 

results \
  .write \
  .partitionBy('Category') \
  .option('orc.compress', 'snappy') \
  .mode('append') \
  .orc("frequencies_orc")

spark.read.orc('frequencies_orc').show()

+---------+---------------+---------+--------+
|ArticleID|            Tag|Frequency|Category|
+---------+---------------+---------+--------+
|     1022|           said|       17|    tech|
|     1022|          firms|       11|    tech|
|     1022|      consumers|       11|    tech|
|     1039|          apple|       12|    tech|
|     1097|         online|       15|    tech|
|      110|        hip-hop|       14|    tech|
|     1111|high-definition|       13|    tech|
|     1111|            dvd|       11|    tech|
|     1114|           maps|       12|    tech|
|     1114|        respond|       11|    tech|
|     1135|            net|       16|    tech|
|     1190|          phone|       15|    tech|
|     1190|      companies|       12|    tech|
|     1190|        premium|       11|    tech|
|     1204|      commodore|       12|    tech|
|     1215|          virus|       12|    tech|
|     1229|         search|       14|    tech|
|     1336|      musicians|       13|    tech|
|     1363|  

Spark SQL & Hive Metastore

In [ ]:
results.createOrReplaceTempView("tags_tmp")

spark.table('tags_tmp').show()

+---------+-------------+----------+---------+
|ArticleID|     Category|       Tag|Frequency|
+---------+-------------+----------+---------+
|       20|         tech|  argonaut|       12|
|       25|         tech|technology|       12|
|       25|         tech|     sound|       11|
|       25|         tech|    mobile|       11|
|       32|         tech|   firefox|       13|
|       46|entertainment|     music|       26|
|       46|entertainment|government|       22|
|       68|entertainment|       bbc|       12|
|       78|        sport|     break|       13|
|       85|entertainment|      film|       11|
|       93|     politics|    answer|       11|
|       93|     politics|      said|       11|
|      110|         tech|   hip-hop|       14|
|      119|        sport|     james|       12|
|      143|         tech|      said|       25|
|      143|         tech|       mac|       19|
|      143|         tech|      mini|       17|
|      148|entertainment|      best|       11|
|      155|  

In [ ]:
spark.sql('show databases').show(10, truncate=False)
spark.sql('show tables in default').show(10, truncate=False)
spark.sql('describe table extended tags_tmp').show(100, truncate=False)

+---------+
|namespace|
+---------+
|bbc      |
|default  |
+---------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |tags_tmp |true       |
+---------+---------+-----------+

+---------+---------+-------+
|col_name |data_type|comment|
+---------+---------+-------+
|ArticleID|int      |null   |
|Category |string   |null   |
|Tag      |string   |null   |
|Frequency|bigint   |null   |
+---------+---------+-------+



In [ ]:
spark.sql('create schema bbc');

results.write.saveAsTable('bbc.results_as_table', format='parquet', partitionBy=['Category', 'Tag'])

AnalysisException: ignored

In [ ]:
spark.sql('show databases').show(10, truncate=False)
spark.sql('show tables in default').show(10, truncate=False)
spark.sql('describe table extended bbc.results_as_table').show(100, truncate=False)

spark.sql('select * from bbc.results_as_table').show(100)

+---------+
|namespace|
+---------+
|bbc      |
|default  |
+---------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |tags_tmp |true       |
+---------+---------+-----------+

+----------------------------+--------------------------------------------------------------+-------+
|col_name                    |data_type                                                     |comment|
+----------------------------+--------------------------------------------------------------+-------+
|ArticleID                   |string                                                        |null   |
|Frequency                   |bigint                                                        |null   |
|Category                    |string                                                        |null   |
|Tag                         |string                                                        |null   |
|# Partition Information     |            

In [ ]:
print(spark.table('bbc.results_as_table').count())

cols = spark.table('bbc.results_as_table').columns 

results.select(*cols).write.insertInto('bbc.results_as_table', overwrite=False)
spark.sql('MSCK repair table bbc.results_as_table')

print(spark.table('bbc.results_as_table').count())

1140
1520


In [ ]:
results.write.option('path', 'bbc/results').saveAsTable('bbc.results_as_external_table', format='parquet')

spark.sql('describe table extended bbc.results_as_external_table').show(100, truncate=False)


AnalysisException: ignored